## using backbone from pre-trained keras resnet50/vgg19 apps

In [1]:
%matplotlib inline
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.applications.resnet50 import *
from tensorflow.keras.datasets import cifar10
# from keras.datasets import cifar10
# from keras.applications.resnet50 import *
# from keras.layers import *
# from keras.models import *
# import keras

import numpy as np
import pandas as pd
keras.__version__

'2.2.4-tf'

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

### let's train a bin classifier, find what happens when pos/neg samples are extremely unbalanced

In [3]:
x_train_bin = x_train.copy()
y_train_bin = y_train==0
x_train_bin.shape, y_train_bin.shape

((50000, 32, 32, 3), (50000, 1))

In [4]:
x = preprocess_input(x_train_bin[0])

In [5]:
x = np.expand_dims(x, axis=0)
x.shape

(1, 32, 32, 3)

In [6]:
m = ResNet50(weights='imagenet', include_top=False)


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [7]:
for layer in m.layers:
    layer.trainable = False

In [8]:
x_features = m.predict(x=x)
x_features.shape

(1, 1, 1, 2048)

In [9]:
x = m.output
x = GlobalAveragePooling2D()(x)
x = Dense(10, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)

In [10]:
baseline_model = Model(inputs=m.input, outputs=prediction)
baseline_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
______________________________________________________________________________________________

In [11]:
baseline_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])

In [12]:
x_valid_bin = x_test.copy()
y_valid_bin = y_test==1

In [13]:
import pandas as pd
hist = baseline_model.fit(x=x_train_bin, y=y_train_bin, validation_data=(x_valid_bin, y_valid_bin), epochs=10, 
                          verbose=1)

W0718 05:59:22.148477 140206562441024 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 42s 844us/sample - loss: 0.2612 - acc: 0.9017 - val_loss: 0.5189 - val_acc: 0.8939
Epoch 2/10
50000/50000 [==============================] - 39s 772us/sample - loss: 0.2301 - acc: 0.9113 - val_loss: 0.5068 - val_acc: 0.8843
Epoch 3/10
50000/50000 [==============================] - 39s 777us/sample - loss: 0.2184 - acc: 0.9158 - val_loss: 0.5024 - val_acc: 0.8809
Epoch 4/10
50000/50000 [==============================] - 39s 778us/sample - loss: 0.2145 - acc: 0.9171 - val_loss: 0.4816 - val_acc: 0.8770
Epoch 5/10
50000/50000 [==============================] - 39s 783us/sample - loss: 0.2075 - acc: 0.9192 - val_loss: 0.4815 - val_acc: 0.8755
Epoch 6/10
50000/50000 [==============================] - 39s 785us/sample - loss: 0.2070 - acc: 0.9206 - val_loss: 0.4886 - val_acc: 0.8789
Epoch 7/10
50000/50000 [==============================] - 39s 785us/sample - loss: 0.2049 - acc: 0.9207 

In [14]:
baseline_model.evaluate(x=x_valid_bin, y=y_valid_bin)

10000/10000 [==============================] - 6s 609us/sample - loss: 0.5260 - acc: 0.8747


[0.5259890766143799, 0.8747]

### train the bin classifier with skewed data

In [15]:
feats = m.output

In [16]:
x = GlobalAveragePooling2D()(feats)
x = Dense(10, activation='relu')(x)
y = Dense(1, activation='sigmoid')(x)
test_model = Model(inputs=m.inputs, outputs=y)

In [17]:
test_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])

In [18]:
mask = (y_train==1)
mask = np.squeeze(mask)
mask.shape

(50000,)

In [19]:
x_train_bin_sess1 = x_train_bin[mask]
x_train_bin_sess1.shape

(5000, 32, 32, 3)

In [20]:
y_train_bin_sess1 = y_train_bin[mask]
y_train_bin_sess1.shape

(5000, 1)

In [21]:
test_model.fit(x=x_train_bin_sess1, y=y_train_bin_sess1, epochs=10, verbose=1)

Train on 5000 samples
Epoch 1/10
5000/5000 [==============================] - 5s 1ms/sample - loss: 0.0686 - acc: 0.9836
Epoch 2/10
5000/5000 [==============================] - 3s 669us/sample - loss: 0.0090 - acc: 1.0000
Epoch 3/10
5000/5000 [==============================] - 3s 668us/sample - loss: 0.0050 - acc: 1.0000
Epoch 4/10
5000/5000 [==============================] - 3s 668us/sample - loss: 0.0036 - acc: 1.0000
Epoch 5/10
5000/5000 [==============================] - 3s 662us/sample - loss: 0.0028 - acc: 1.0000
Epoch 6/10
5000/5000 [==============================] - 3s 664us/sample - loss: 0.0023 - acc: 1.0000
Epoch 7/10
5000/5000 [==============================] - 3s 664us/sample - loss: 0.0019 - acc: 1.0000
Epoch 8/10
5000/5000 [==============================] - 3s 663us/sample - loss: 0.0017 - acc: 1.0000
Epoch 9/10
5000/5000 [==============================] - 3s 662us/sample - loss: 0.0014 - acc: 1.0000
Epoch 10/10
5000/5000 [==============================] - 3s 662us/sampl

In [22]:
test_model.evaluate(x=x_valid_bin, y=y_valid_bin)

10000/10000 [==============================] - 7s 722us/sample - loss: 1.5358 - acc: 0.9000


[1.5357855224671462, 0.9]

Appearantly, that wont work since this model is only taught by possitive examples. An auto negative example gens seems to be necessary. Just keep stream in negative samples and see what happens

In [23]:
# this time stream in data with 2nd class
mask = y_train==2
mask = np.squeeze(mask)
x_train_bin_sess2 = x_train_bin[mask]
y_train_bin_sess2 = y_train_bin[mask]
mask.shape, x_train_bin_sess2.shape, y_train_bin_sess2.shape

((50000,), (5000, 32, 32, 3), (5000, 1))

In [24]:
test_model.fit(x=x_train_bin_sess2, y=y_train_bin_sess2, epochs=10, verbose=1)

Train on 5000 samples
Epoch 1/10
5000/5000 [==============================] - 3s 661us/sample - loss: 0.0013 - acc: 1.0000
Epoch 2/10
5000/5000 [==============================] - 3s 660us/sample - loss: 0.0013 - acc: 1.0000
Epoch 3/10
5000/5000 [==============================] - 3s 659us/sample - loss: 0.0011 - acc: 1.0000
Epoch 4/10
5000/5000 [==============================] - 3s 660us/sample - loss: 0.0010 - acc: 1.0000
Epoch 5/10
5000/5000 [==============================] - 3s 660us/sample - loss: 0.0010 - acc: 1.0000
Epoch 6/10
5000/5000 [==============================] - 3s 660us/sample - loss: 7.8960e-04 - acc: 1.0000
Epoch 7/10
5000/5000 [==============================] - 3s 660us/sample - loss: 8.0293e-04 - acc: 1.0000
Epoch 8/10
5000/5000 [==============================] - 3s 663us/sample - loss: 7.5139e-04 - acc: 1.0000
Epoch 9/10
5000/5000 [==============================] - 3s 660us/sample - loss: 6.7300e-04 - acc: 1.0000
Epoch 10/10
5000/5000 [==============================

In [25]:
testout = test_model.predict(x_valid_bin)

The model simply goes to the other extreme end

In [26]:
np.sum(testout>0.1)

0

Even in bin classifier, skewed data hurt model badly

## Train the bin classifier incrementally

Next, we create 'perfect' learning sessions manually trying to defeat the baseline model (89% valid acc). The purpose of this experiment is to figure out whether extra negtive samples help with the model

In [27]:
def get_model_bin_classify(feature_extractor):
    x = feature_extractor.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(10, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=feature_extractor.input, outputs=x)
    model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])
    return model

In [28]:
def get_train_samples(X_train, y_train, cls_li):
    mask = np.zeros(y_train.shape, dtype=np.bool)
    for cls in cls_li:
        mask += (y_train==cls)
    mask = np.squeeze(mask)
    return X_train[mask], y_train[mask]
    
def eva_bin_on_cls(model, X_valid, y_valid, cls_li, pos_samples):
    for cls in cls_li:
        mask = (y_valid==cls)
        mask = np.squeeze(mask)
        X_valid_cur_sess = X_valid[mask]
        y_valid_cur_sess = y_valid[mask]
        y_valid_cur_sess = convert_to_bin_samples(y_valid_cur_sess, pos_samples)
        loss, metric = model.evaluate(x=X_valid_cur_sess, y=y_valid_cur_sess)
        print('Evaluation loss: %f. Acc: %f' % (loss, metric))

def convert_to_bin_samples(labels, pos_samples):
    # arg1 = original labels, pos_samples = samples mark possitive
    re_label = np.zeros(labels.shape, dtype=np.bool)
    for it in pos_samples:
        re_label += (labels == it)
    return re_label

In [29]:
feat_ext = ResNet50(weights='imagenet', include_top=False)

In [30]:
test_model = get_model_bin_classify(feat_ext)

In [31]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 1])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 2s 2ms/sample - loss: 0.1441 - acc: 0.9500
Evaluation loss: 0.144050. Acc: 0.950000
1000/1000 [==============================] - 1s 633us/sample - loss: 0.0821 - acc: 0.9670
Evaluation loss: 0.082093. Acc: 0.967000


In [32]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 2])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 1s 624us/sample - loss: 0.2400 - acc: 0.9190
Evaluation loss: 0.239962. Acc: 0.919000
1000/1000 [==============================] - 1s 628us/sample - loss: 1.5052 - acc: 0.6150
Evaluation loss: 1.505151. Acc: 0.615000
1000/1000 [==============================] - 1s 631us/sample - loss: 0.1875 - acc: 0.9460
Evaluation loss: 0.187515. Acc: 0.946000


In [33]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 3])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2, 3], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 1s 625us/sample - loss: 0.2062 - acc: 0.9410
Evaluation loss: 0.206238. Acc: 0.941000
1000/1000 [==============================] - 1s 633us/sample - loss: 1.5682 - acc: 0.6860
Evaluation loss: 1.568209. Acc: 0.686000
1000/1000 [==============================] - 1s 631us/sample - loss: 0.5156 - acc: 0.8830
Evaluation loss: 0.515635. Acc: 0.883000
1000/1000 [==============================] - 1s 633us/sample - loss: 0.0714 - acc: 0.9770
Evaluation loss: 0.071355. Acc: 0.977000


In [34]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 4])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2, 3, 4], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 1s 624us/sample - loss: 0.1586 - acc: 0.9540
Evaluation loss: 0.158594. Acc: 0.954000
1000/1000 [==============================] - 1s 629us/sample - loss: 2.0247 - acc: 0.6100
Evaluation loss: 2.024703. Acc: 0.610000
1000/1000 [==============================] - 1s 627us/sample - loss: 0.7766 - acc: 0.8310
Evaluation loss: 0.776602. Acc: 0.831000
1000/1000 [==============================] - 1s 625us/sample - loss: 0.1921 - acc: 0.9460
Evaluation loss: 0.192053. Acc: 0.946000
1000/1000 [==============================] - 1s 626us/sample - loss: 0.0819 - acc: 0.9770
Evaluation loss: 0.081864. Acc: 0.977000


In [35]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 5])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2, 3, 4, 5], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 1s 619us/sample - loss: 0.1567 - acc: 0.9610
Evaluation loss: 0.156739. Acc: 0.961000
1000/1000 [==============================] - 1s 622us/sample - loss: 2.3434 - acc: 0.5570
Evaluation loss: 2.343359. Acc: 0.557000
1000/1000 [==============================] - 1s 627us/sample - loss: 0.9369 - acc: 0.8320
Evaluation loss: 0.936868. Acc: 0.832000
1000/1000 [==============================] - 1s 627us/sample - loss: 0.1306 - acc: 0.9670
Evaluation loss: 0.130578. Acc: 0.967000
1000/1000 [==============================] - 1s 626us/sample - loss: 0.1863 - acc: 0.9530
Evaluation loss: 0.186348. Acc: 0.953000
1000/1000 [==============================] - 1s 635us/sample - loss: 0.0531 - acc: 0.9830
Evaluation loss: 0.053126. Acc: 0.983000


In [36]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 6])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2, 3, 4, 5, 6], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 1s 612us/sample - loss: 0.2391 - acc: 0.9470
Evaluation loss: 0.239109. Acc: 0.947000
1000/1000 [==============================] - 1s 614us/sample - loss: 1.5170 - acc: 0.7050
Evaluation loss: 1.516992. Acc: 0.705000
1000/1000 [==============================] - 1s 617us/sample - loss: 0.7591 - acc: 0.8680
Evaluation loss: 0.759094. Acc: 0.868000
1000/1000 [==============================] - 1s 617us/sample - loss: 0.1577 - acc: 0.9570
Evaluation loss: 0.157724. Acc: 0.957000
1000/1000 [==============================] - 1s 617us/sample - loss: 0.1820 - acc: 0.9520
Evaluation loss: 0.182013. Acc: 0.952000
1000/1000 [==============================] - 1s 619us/sample - loss: 0.0766 - acc: 0.9790
Evaluation loss: 0.076595. Acc: 0.979000
1000/1000 [==============================] - 1s 619us/sample - loss: 0.0247 - acc: 0.9900
Evaluation loss: 0.024733. Acc: 0.990000


In [37]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 7])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2, 3, 4, 5, 6, 7], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 1s 616us/sample - loss: 0.2057 - acc: 0.9480
Evaluation loss: 0.205710. Acc: 0.948000
1000/1000 [==============================] - 1s 617us/sample - loss: 1.8269 - acc: 0.6500
Evaluation loss: 1.826942. Acc: 0.650000
1000/1000 [==============================] - 1s 615us/sample - loss: 0.8972 - acc: 0.8460
Evaluation loss: 0.897247. Acc: 0.846000
1000/1000 [==============================] - 1s 615us/sample - loss: 0.1639 - acc: 0.9520
Evaluation loss: 0.163897. Acc: 0.952000
1000/1000 [==============================] - 1s 614us/sample - loss: 0.1577 - acc: 0.9580
Evaluation loss: 0.157662. Acc: 0.958000
1000/1000 [==============================] - 1s 612us/sample - loss: 0.0551 - acc: 0.9790
Evaluation loss: 0.055144. Acc: 0.979000
1000/1000 [==============================] - 1s 617us/sample - loss: 0.0887 - acc: 0.9770
Evaluation loss: 0.088687. Acc: 0.977000
1000/1000 

In [38]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 8])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2, 3, 4, 5, 6, 7, 8], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 1s 616us/sample - loss: 0.2981 - acc: 0.9130
Evaluation loss: 0.298082. Acc: 0.913000
1000/1000 [==============================] - 1s 617us/sample - loss: 0.5979 - acc: 0.8270
Evaluation loss: 0.597911. Acc: 0.827000
1000/1000 [==============================] - 1s 618us/sample - loss: 1.5912 - acc: 0.6320
Evaluation loss: 1.591243. Acc: 0.632000
1000/1000 [==============================] - 1s 615us/sample - loss: 0.6739 - acc: 0.8200
Evaluation loss: 0.673856. Acc: 0.820000
1000/1000 [==============================] - 1s 616us/sample - loss: 1.1033 - acc: 0.7360
Evaluation loss: 1.103252. Acc: 0.736000
1000/1000 [==============================] - 1s 613us/sample - loss: 0.5316 - acc: 0.8410
Evaluation loss: 0.531591. Acc: 0.841000
1000/1000 [==============================] - 1s 615us/sample - loss: 0.6437 - acc: 0.8260
Evaluation loss: 0.643745. Acc: 0.826000
1000/1000 

In [39]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 9])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 1s 615us/sample - loss: 0.5669 - acc: 0.8810
Evaluation loss: 0.566950. Acc: 0.881000
1000/1000 [==============================] - 1s 619us/sample - loss: 0.0944 - acc: 0.9760
Evaluation loss: 0.094372. Acc: 0.976000
1000/1000 [==============================] - 1s 618us/sample - loss: 1.6673 - acc: 0.6880
Evaluation loss: 1.667345. Acc: 0.688000
1000/1000 [==============================] - 1s 616us/sample - loss: 0.3308 - acc: 0.9170
Evaluation loss: 0.330831. Acc: 0.917000
1000/1000 [==============================] - 1s 616us/sample - loss: 0.5432 - acc: 0.8540
Evaluation loss: 0.543224. Acc: 0.854000
1000/1000 [==============================] - 1s 616us/sample - loss: 0.2011 - acc: 0.9460
Evaluation loss: 0.201109. Acc: 0.946000
1000/1000 [==============================] - 1s 616us/sample - loss: 0.1399 - acc: 0.9660
Evaluation loss: 0.139904. Acc: 0.966000
1000/1000 

The result is rather satisfied yet confusing. Why the perf on 2nd class differs by turns? Let's see how a partial trained bin classifier perform on unseened data

## Experiments on unseen negative samples

In [40]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 1])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=1)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2, 3, 4, 5], [0])

Train on 10000 samples
Epoch 1/5
10000/10000 [==============================] - 26s 3ms/sample - loss: 0.0284 - acc: 0.9902
Epoch 2/5
10000/10000 [==============================] - 26s 3ms/sample - loss: 0.0097 - acc: 0.9966
Epoch 3/5
10000/10000 [==============================] - 26s 3ms/sample - loss: 0.0063 - acc: 0.9980
Epoch 4/5
10000/10000 [==============================] - 26s 3ms/sample - loss: 0.0040 - acc: 0.9986
Epoch 5/5
1000/1000 [==============================] - 1s 617us/sample - loss: 0.3579 - acc: 0.9270
Evaluation loss: 0.357893. Acc: 0.927000
1000/1000 [==============================] - 1s 614us/sample - loss: 0.0515 - acc: 0.9860
Evaluation loss: 0.051527. Acc: 0.986000
1000/1000 [==============================] - 1s 614us/sample - loss: 2.7299 - acc: 0.5450
Evaluation loss: 2.729926. Acc: 0.545000
1000/1000 [==============================] - 1s 616us/sample - loss: 0.7109 - acc: 0.8500
Evaluation loss: 0.710879. Acc: 0.850000
1000/1000 [============================

In [41]:
X_test_subset, Y_test_set = get_train_samples(x_test, y_test, [2])
prediction = test_model.predict(X_test_subset)

In [42]:
np.sum(prediction < 0.9) / np.squeeze(prediction).shape[0]

0.643

The model **does not generate itself well enough on unseen negative samples** even by increasing the threshold up to 0.9, yet it will **degrade its performance on positive samples sufficiently**. By which, every/relavent classifiers need to be updated by turns.

## conclusion
- Performance on seen negative samples will be intefered when newly arrived negative samples observed by model. Sometimes increasing sometimes decreasing. 
- It seems like that the model's generation capability is performing better and better when enough negative classes have been trained before.
- The model performs poorly on unseen negative samples which emphasizes the necessity of upgrade when new negative class came